In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

In [3]:
dataset_airports = pd.read_csv("Airports Database.csv")
dataset_ext = pd.read_csv("External Flights Database.csv")
dataset_fleet = pd.read_csv("Fleet Database.csv")
dataset_load = pd.read_csv("Load Database.csv")
load = pd.MultiIndex.from_frame(dataset_load)
airports = pd.MultiIndex.from_frame(dataset_airports)
fleet = pd.MultiIndex.from_frame(dataset_fleet)
ext = pd.MultiIndex.from_frame(dataset_ext)

In [4]:
A = list(dataset_airports["ID"])
#HUB = [i for i in A if dataset_airports["HUB"][i-1]==1]
HUB = [3]
N = list(dataset_fleet["ID"])
L = list(dataset_load["ID"])
E = list(dataset_ext["ID"])
T = [i for i in np.arange(7)+1]
MNT = [0 for i in N]
for i in N:
    MNT[i-1] = [dataset_fleet["MNT 1"][i-1],dataset_fleet["MNT 2"][i-1]]
#max_cycles = int(input("\nEnter maximum number of intermediaries between origin and destination of cargo : "))
max_cycles = 3
T = 7320

In [5]:
# Generating Cargo Flow Paths for all OD pairs

def cargo_flow_path(b,ix):
    list = []
    
    def f(lt,ix):
        if ix == 0:
            list.append(lt.copy()) 
        else :
            for j in b:
                if len(lt) == 0:
                    lt2 = lt.copy()
                    lt2.append(j)
                    f(lt2.copy(),ix-1)
                elif j not in lt:
                    lt2 = lt.copy()
                    lt2.append(j)
                    f(lt2.copy(),ix-1)
                    
    f(list.copy(),ix)
    return list
                

def cargo_path_generator(orig,dest):
    b = [i for i in A if i not in [orig,dest]]
    paths = []
    paths.append([orig,dest])
    for i in range(min(max_cycles,len(A)-2)):
        P = cargo_flow_path(b,i+1)
        for p in P:
            list = []
            list.append(orig)
            for z in p:
                list.append(z)
            list.append(dest)
            paths.append(list)
            
    return paths

In [6]:
# Generating Routes for all Planes in Fleet

def range_feasible(j,id,orig) :
    if dataset_airports[str(j)][orig-1]<dataset_fleet.Range[id-1]:
        return True
    else :
        return False
    
def mnt_feasible(j,id,t,a) :
    if dataset_airports[str(j)][a-1]/dataset_fleet.Speed[id-1]*60 + t < dataset_fleet.Uptime[id-1]:
        return True
    else :
        return False
    
def aircraft_route(orig,ix,id):
    list = []
    
    def f(lt,ix,t):
        if ix == 0:
            list.append(lt.copy()) 
        else :
            for j in A:
                if len(lt) == 0:
                    if j != orig and range_feasible(j,id,orig) and mnt_feasible(j,id,t,orig):
                        lt2 = lt.copy()
                        lt2.append(j)
                        if j not in MNT[id-1]:
                            t = t + dataset_airports[str(j)][orig-1]/dataset_fleet.Speed[id-1]
                        else :
                            t = 0
                        f(lt2.copy(),ix-1,t)
                        
                elif j != lt[-1]:
                    if ix == 1 and j == orig :
                        continue
                    elif range_feasible(j,id,orig) and mnt_feasible(j,id,t,lt[-1]):
                        lt2 = lt.copy()
                        lt2.append(j)
                        if j not in MNT[id-1]:
                            t = t + dataset_airports[str(j)][lt[-1]-1]/dataset_fleet.Speed[id-1]*60
                        else :
                            t = 0
                        f(lt2.copy(),ix-1,t)
                    
    f(list.copy(),ix,0)
    return list

def route(id) :
    paths = []
    orig = dataset_fleet.Origin[id-1]
    
    for i in range(min(7,dataset_fleet["Max Cycles"][id-1])) :
        if i==0 :
            continue
        P = aircraft_route(orig,i,id)
        for p in P:
            list = []
            list.append(orig)
            for z in p:
                list.append(z)
            if i != 0:
                list.append(orig)
            paths.append(list)
    
    return paths

In [7]:
routes = [[] for j in N]
for i in N:
    for r in route(i):
        routes[i-1].append(r)
route_len = [list(np.arange(len(r))) for r in routes]

In [8]:
paths = [[[] for i in A] for j in A]
for i in A:
    for j in A:
        if i==j:
            continue
        else :
            paths[i-1][j-1] = cargo_path_generator(i,j)

In [9]:
S = [[] for i in L]
for i in L:
    origin = load[i-1][1]
    destination = load[i-1][2]
    for p in paths[origin-1][destination-1]:
        S[i-1].append(p)
S_len = [list(np.arange(len(s))) for s in S]
S

[[[1, 2], [1, 3, 2], [1, 4, 2], [1, 3, 4, 2], [1, 4, 3, 2]],
 [[4, 2], [4, 1, 2], [4, 3, 2], [4, 1, 3, 2], [4, 3, 1, 2]],
 [[4, 3], [4, 1, 3], [4, 2, 3], [4, 1, 2, 3], [4, 2, 1, 3]],
 [[2, 3], [2, 1, 3], [2, 4, 3], [2, 1, 4, 3], [2, 4, 1, 3]]]

In [10]:
mdl = Model('Aircraft Scheduling')

P = [(mdl.addVar(vtype = GRB.BINARY, name="P%s" % str([index,k])
               .format(index,k))) for index in L for k in S_len[index-1]]

B = [(mdl.addVar(vtype = GRB.BINARY, name="B%s" % str([i,u])
                .format(i,u))) 
                     for i in N for u in route_len[i-1]]

X = [(mdl.addVar(vtype = GRB.BINARY, lb = 0.0, name="X%s" % str([i,index,k,leg])
                 .format(i,index,k,leg))) 
                     for index in L for i in N for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])] 

Z = [(mdl.addVar(vtype = GRB.BINARY, lb = 0.0, name="Z%s" % str([j,index,k,leg])
                 .format(j,index,k,leg))) 
                     for index in L for j in E for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])] 

Tdepc = [(mdl.addVar(vtype = GRB.INTEGER, name="Tdepc%s" % str([index,k,leg])
               .format(index,k,leg), lb = 0.0)) 
                    for index in L  for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])]

Tarrc = [(mdl.addVar(vtype = GRB.INTEGER, name="Tarrc%s" % str([index,k,leg])
               .format(index,k,leg), lb = 0.0)) 
                    for index in L  for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][1:])]

Tdepa = [(mdl.addVar(vtype = GRB.INTEGER, name="Tdepa%s" % str([i,k,leg])
               .format(i,k,leg), lb = 0.0)) 
                    for i in N  for k in route_len[i-1] for leg,_ in enumerate(routes[i-1][k][:-1])]

Tarra = [(mdl.addVar(vtype = GRB.INTEGER, name="Tarra%s" % str([i,k,leg])
               .format(i,k,leg), lb = 0.0)) 
                    for i in N  for k in route_len[i-1] for leg,_ in enumerate(routes[i-1][k][1:])]

Tacc = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tacc%s" % str([i,k,leg])
               .format(i,k,leg), lb = 0.0)) 
                    for i in N  for k in route_len[i-1] for leg,_ in enumerate(routes[i-1][k])]

Tl = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Tl%s" % str([index])
                 .format(index), lb = 0.0))
                     for index in L]

Y = [(mdl.addVar(vtype = GRB.BINARY, name="Y%s" % str([i,k,leg])
                .format(i,k,leg))) 
                     for i in N for k in route_len[i-1] for leg,_ in enumerate(routes[i-1][k][:-1])] 

F = [(mdl.addVar(vtype = GRB.BINARY, name="F%s" % str([i,u,leg,index,k,leg_c])
                .format(i,u,leg,index,k,leg_c))) 
                     for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])
                     for index in L for k in S_len[index-1] for leg_c,_ in enumerate(S[index-1][k][:-1])] 

Ctrf = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Ctrf%s" % str([index])
                   .format(index), lb = 0.0))
                   for index in L]

Cfix = [(mdl.addVar(vtype = GRB.CONTINUOUS,name="Cfix%s" % str([i])
                   .format(i), lb = 0.0))
                   for i in N]

Cvar = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cvar%s" % str([i])
                   .format(i), lb = 0.0))
                   for i in N]

Cext = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cext%s" % str([j])
                   .format(j), lb = 0.0))
                   for j in E]

Cmnt = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cmnt%s" % str([i])
                   .format(i), lb = 0.0))
                   for i in N]

Cgrd = [(mdl.addVar(vtype = GRB.CONTINUOUS, name="Cgrd%s" % str([i])
                   .format(i), lb = 0.0))
                   for i in N]

mdl.update()

mdl.modelSense = GRB.MINIMIZE

mdl.setObjectiveN((sum(mdl.getVarByName("Ctrf"+str([index])) for index in L)
                + sum(mdl.getVarByName("Cfix"+str([i])) for i in N) 
                + sum(mdl.getVarByName("Cgrd"+str([i])) for i in N)
                + sum(mdl.getVarByName("Cvar"+str([i])) for i in N)
                + sum(mdl.getVarByName("Cext"+str([j])) for j in E) 
                + sum(mdl.getVarByName("Cmnt"+str([i])) for i in N)),0, 1)

mdl.setObjectiveN((sum(mdl.getVarByName("Tl"+str([index])) for index in L)),1,0)


Academic license - for non-commercial use only - expires 2021-12-18
Using license file C:\Users\Rishav\gurobi.lic


In [11]:
#CONSTRAINTS FOR BINARY VARIABLES AND INITIALISATIONS

mdl.addConstrs((quicksum(mdl.getVarByName("P"+str([index,k])) 
                    for k in S_len[index-1]) == 1 for index in L), name = "One_Path");

mdl.addConstrs((quicksum(mdl.getVarByName("B"+str([i,u])) 
                    for u in route_len[i-1]) <= 1 for i in N), name = "At_max_One_Route");

mdl.addConstrs(((
    sum(mdl.getVarByName("X"+str([i,index,k,leg])) for i in N) + 
    sum(mdl.getVarByName("Z"+str([j,index,k,leg])) for j in E)) 
    == mdl.getVarByName("P"+str([index,k]))
        for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]) ), name = "Only_one_plane_or_no_plane"); 


mdl.addConstrs((
    mdl.getVarByName("X"+str([i,index,k,leg]))
    <= sum(mdl.getVarByName("B"+str([i,u])) for u in route_len[i-1])
        for index in L for i in N for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]) ), name = "Route_has_to_be_Selected");


mdl.addConstrs((
    mdl.getVarByName("Tdepc"+str([index,k,leg]))
    <= T*mdl.getVarByName("P"+str([index,k])) 
    for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])), name="Getting_departure_times_to_0_C");

mdl.addConstrs((
    mdl.getVarByName("Tarrc"+str([index,k,leg]))
    <= T*mdl.getVarByName("P"+str([index,k])) 
    for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][1:])), name="Getting_arrival_times_to_0_C");

mdl.addConstrs((
    mdl.getVarByName("Tdepa"+str([i,u,leg]))
    <= T*mdl.getVarByName("B"+str([i,u])) 
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name="Getting_departure_times_to_0_P");

mdl.addConstrs((
    mdl.getVarByName("Tarra"+str([i,u,leg]))
    <= T*mdl.getVarByName("B"+str([i,u])) 
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][1:])), name="Getting_arrival_times_to_0_P");

mdl.addConstrs((
    mdl.getVarByName("Tacc"+str([i,u,0])) == 0
    for i in N for u in route_len[i-1] ), name = "Accumulated_time_plane_to_0");

mdl.addConstrs((
    mdl.getVarByName("Tacc"+str([i,u,leg]))
    <= fleet[i-1][6]*(1 - mdl.getVarByName("Y"+str([i,u,leg])))
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name="Getting_accumulated_times_to_0_P");

#mdl.addConstrs((
#    mdl.getVarByName("Y"+str([i,u,0])) == 1
#    for i in N for u in route_len[i-1]), name = "First_is_MNT");

mdl.addConstrs((
    mdl.getVarByName("Tl"+str([index])) >= (sum(mdl.getVarByName("Tarrc"+str([index,k,len(S[index-1][k])-2])) for k in S_len[index-1]) - load[index-1][5])
    for index in L ), name = "Time_Delivered");

In [12]:
#CARGO CAN JUMP SHIP ONLY AT TRANSFER HUBS

mdl.addConstrs((
    mdl.getVarByName("X"+str([i,index,k,leg])) 
    <= mdl.getVarByName("X"+str([i,index,k,leg+1]))
    for i in N for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-2]) 
    if S[index-1][k][leg+1] not in HUB), name = "Transfer_Hubs");

In [13]:
#EXTERNAL FLIGHTS MATCH THE REQUIREMENTS OF THE CARGO

mdl.addConstrs((
    mdl.getVarByName("Z"+str([j,index,k,leg])) == 0
    for j in E for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]) 
    if ext[j-1][7] != S[index-1][k][leg] or ext[j-1][8] != S[index-1][k][leg+1]), 
    name = "Ext_legs_not_match");

#mdl.addConstrs((
#    mdl.getVarByName("Z"+str([j,index,k,leg])) <= 1
#    for j in E for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]) 
#    if ext[j-1][7] == S[index-1][k][leg] or ext[j-1][8] == S[index-1][k][leg+1]), 
#    name = "Ext_legs_match");

mdl.addConstrs((
    (mdl.getVarByName("Z"+str([j,index,k,leg])) == 1) >> 
    (mdl.getVarByName("Tdepc"+str([index,k,leg])) == ext[j-1][9])
    for index in L for j in E for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]) ), name = "Time_match_ext_dep");

mdl.addConstrs((
    (mdl.getVarByName("Z"+str([j,index,k,leg])) == 1) >> 
    (mdl.getVarByName("Tarrc"+str([index,k,leg])) == ext[j-1][10])
    for index in L for j in E for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][1:]) ), name = "Time_match_ext_arr");

mdl.addConstrs((
    sum(mdl.getVarByName("Z"+str([j,index,k,leg])) * 
    load[index-1][3] for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]))
    <= ext[j-1][3] 
    for j in E), name = "Weight_constraint_ext");

In [14]:
#TIME CONSTRAINTS FOR CARGO PATHS

mdl.addConstrs((
    mdl.getVarByName("Tdepc"+str([index,k,0]))
    >= (load[index-1][4]*mdl.getVarByName("P"+str([index,k]))  + 
        sum(mdl.getVarByName("X"+str([i,index,k,0]))
            *dataset_airports["gt"+str(fleet[i-1][2])][S[index-1][k][0]] for i in N))
    for index in L for k in S_len[index-1]), name = "First_departure_time_cargo");

mdl.addConstrs((
    mdl.getVarByName("Tdepc"+str([index,k,leg+1]))
    >= (mdl.getVarByName("Tarrc"+str([index,k,leg])) + 
        sum(mdl.getVarByName("X"+str([i,index,k,leg+1]))
            *(dataset_airports["gt"+str(fleet[i-1][2])][S[index-1][k][leg+1]]) for i in N))
    for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][1:-1])), name = "Departure_time_cargo");

mdl.addConstrs((
    mdl.getVarByName("Tarrc"+str([index,k,leg])) >= mdl.getVarByName("Tdepc"+str([index,k,leg])) + 
    sum(mdl.getVarByName("X"+str([i,index,k,leg])) * (dataset_airports[str(S[index-1][k][leg]+1)][S[index-1][k][leg+1]]
                  /fleet[i-1][4]*60) for i in N)
    for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])), name = "Arrival_time_cargo1");

mdl.addConstrs((
    mdl.getVarByName("Tarrc"+str([index,k,leg])) <= mdl.getVarByName("Tdepc"+str([index,k,leg])) + 1 + 1000*(1-sum(mdl.getVarByName("X"+str([i,index,k,leg])) for i in N)) + 
    sum(mdl.getVarByName("X"+str([i,index,k,leg])) * (dataset_airports[str(S[index-1][k][leg]+1)][S[index-1][k][leg+1]]
                  /fleet[i-1][4]*60) for i in N)
    for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])), name = "Arrival_time_cargo2");

In [15]:
#TIME CONSTRAINTS FOR PLANE ROUTES

mdl.addConstrs((
    mdl.getVarByName("Tarra"+str([i,u,leg])) >= mdl.getVarByName("Tdepa"+str([i,u,leg])) + 
    mdl.getVarByName("B"+str([i,u]))
            *(dataset_airports[str(routes[i-1][u][leg]+1)][routes[i-1][u][leg+1]]
                  /fleet[i-1][4]*60)
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name = "Arrival_time_plane_0");

mdl.addConstrs((
    mdl.getVarByName("Tarra"+str([i,u,leg])) <= mdl.getVarByName("Tdepa"+str([i,u,leg])) + 1 +
    mdl.getVarByName("B"+str([i,u]))
            *(dataset_airports[str(routes[i-1][u][leg]+1)][routes[i-1][u][leg+1]]
                  /fleet[i-1][4]*60)
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name = "Arrival_time_plane_1");

mdl.addConstrs((
    mdl.getVarByName("Tdepa"+str([i,u,leg+1])) >= mdl.getVarByName("Tarra"+str([i,u,leg])) +
    mdl.getVarByName("B"+str([i,u]))
            *(dataset_airports["gt"+str(fleet[i-1][2])][routes[i-1][u][leg+1]])
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-2])), name = "Arrival_time_plane");


mdl.addConstrs((
    
    mdl.getVarByName("Tacc"+str([i,u,leg+1])) >= (mdl.getVarByName("Tacc"+str([i,u,leg])) + 
                                                  dataset_airports[str(routes[i-1][u][leg]+1)][routes[i-1][u][leg+1]]/fleet[i-1][4]*60) -  
                                                 mdl.getVarByName("Y" + str([i,u,leg+1])) * T
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][1:-1])), name = "Accumulated_time_plane");

mdl.addConstrs((
    mdl.getVarByName("Y" + str([i,u,leg])) <= mdl.getVarByName("B" + str([i,u]))
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name = "Mnt1");

mdl.addConstrs((
    mdl.getVarByName("Y" + str([i,u,leg])) <= 1
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1]) if routes[i-1][u][leg] in MNT[i-1]), name = "Mnt2");

mdl.addConstrs((
    mdl.getVarByName("Y" + str([i,u,leg])) <= 0
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1]) if routes[i-1][u][leg] not in MNT[i-1]), name = "Mnt3");

mdl.addConstrs((
    (mdl.getVarByName("Tacc"+str([i,u,leg+1])) + 
     dataset_airports[str(routes[i-1][u][leg+1]+1)][routes[i-1][u][leg+2]]/fleet[i-1][4]*60 - 
     fleet[i-1][6] 
    ) *(1- mdl.getVarByName("Y" + str([i,u,leg+1]))) <= 0
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][1:-1])), name = "Accumulated_time_plane_2");

In [16]:
#ATTACHING FLIGHT ROUTES AND CARGO FLOW PATHS IN A VARIABLE

mdl.addConstrs((
    sum(
        mdl.getVarByName("F"+str([i,u,leg,index,k,leg_c])) 
        for leg,_ in enumerate(routes[i-1][u][:-1]) 
        if S[index-1][k][leg_c] == routes[i-1][u][leg] and S[index-1][k][leg_c+1] == routes[i-1][u][leg+1]
    )  == 
    mdl.getVarByName("B" + str([i,u])) * 
    mdl.getVarByName("X"+str([i,index,k,leg_c]))
    for i in N for u in route_len[i-1]
    for index in L for k in S_len[index-1] for leg_c,_ in enumerate(S[index-1][k][:-1]) ),name = "F1");

mdl.addConstrs((
    (mdl.getVarByName("F"+str([i,u,leg,index,k,leg_c])) == 1) >> 
    (mdl.getVarByName("Tarra"+str([i,u,leg])) == mdl.getVarByName("Tarrc"+str([index,k,leg_c])))
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])
    for index in L for k in S_len[index-1] for leg_c,_ in enumerate(S[index-1][k][:-1])), name = "F2");

mdl.addConstrs((
    (mdl.getVarByName("F"+str([i,u,leg,index,k,leg_c])) == 1) >> 
    (mdl.getVarByName("Tdepa"+str([i,u,leg])) == mdl.getVarByName("Tdepc"+str([index,k,leg_c])))
    for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])
    for index in L for k in S_len[index-1] for leg_c,_ in enumerate(S[index-1][k][:-1])), name = "F3");

mdl.addConstrs((
    mdl.getVarByName("F"+str([i,u,leg2,index,k,leg_c2])) <= 1 - mdl.getVarByName("F"+str([i,u,leg1,index,k,leg_c1]))
    
    for i in N for u in route_len[i-1] 
    for leg1,_ in enumerate(routes[i-1][u][:-1]) for leg2,_ in enumerate(routes[i-1][u][:-1]) 
    for index in L for k in S_len[index-1] 
    for leg_c1,_ in enumerate(S[index-1][k][:-1]) for leg_c2,_ in enumerate(S[index-1][k][:-1]) if leg1>leg2 and leg_c1<leg_c2), name = "F4");

In [17]:
#WEIGHT OF CARGO IS NOT MORE THAN FLIGHT CAPACITY

mdl.addConstrs((
    sum(
        mdl.getVarByName("F"+str([i,u,leg,index,k,leg_c])) * 
        load[index-1][3] 
        for index in L for k in S_len[index-1] for leg_c,_ in enumerate(S[index-1][k][:-1])
    )
    <= fleet[i-1][3] 
     for i in N for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])), name = "Weight_of_fleet");

In [18]:
#Transfering Costs
mdl.addConstrs((mdl.getVarByName("Ctrf"+str([index]))
               ==sum(mdl.getVarByName("X"+str([i,index,k,leg]))
                       * mdl.getVarByName("X"+str([g,index,k,leg+1]))
                       * (dataset_airports[str(fleet[i-1][2])][S[index-1][k][leg+1]]+dataset_airports[str(fleet[g-1][2])][S[index-1][k][leg+1]])
                       for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-2]) for i in N for g in N if i!=g) 
                for index in L ),name="Transfer_costs");

#Fixed Costs of Routes
mdl.addConstrs((mdl.getVarByName("Cfix"+str([i])) == 
                sum(mdl.getVarByName("B"+str([i,u])) * 
               (sum(dataset_airports[str(routes[i-1][u][leg]+1)][routes[i-1][u][leg+1]]
                   *fleet[i-1][11]    
               for leg,_ in enumerate(routes[i-1][u][:-1])) + fleet[i-1][10]) for u in route_len[i-1]) for i in N ), name = "fixed_costs");

#Variable Costs of Routes
mdl.addConstrs((mdl.getVarByName("Cvar"+str([i])) ==
                sum(mdl.getVarByName("X"+str([i,index,k,leg])) * 
                    fleet[i-1][17] * load[index-1][3] * dataset_airports[str(S[index-1][k][leg]+1)][S[index-1][k][leg+1]] 
                  for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])) for i in N ), name = "variable_costs");

#Costs for external ships
mdl.addConstrs((mdl.getVarByName("Cext"+str([j])) == sum(mdl.getVarByName("Z"+str([j,index,k,leg])) for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1]))*ext[j-1][4]+
               sum(mdl.getVarByName("Z"+str([j,index,k,leg]))*ext[j-1][6]*load[index-1][3] 
                for index in L for k in S_len[index-1] for leg,_ in enumerate(S[index-1][k][:-1])) for j in E), name = "External_costs");

#Costs for Maintenance
mdl.addConstrs((mdl.getVarByName("Cmnt"+str([i])) == 
              sum(mdl.getVarByName("Y" + str([i,u,leg])) * dataset_airports[str(fleet[i-1][2])][routes[i-1][u][leg]] 
                 for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-1])) for i in N), name = "Maintenance_costs");

#Costs for Grounding
mdl.addConstrs((mdl.getVarByName("Cgrd"+str([i])) == 
              sum((mdl.getVarByName("Tdepa" + str([i,u,leg+1]))-mdl.getVarByName("Tarra" + str([i,u,leg])))
                  * dataset_airports["gr" + str(fleet[i-1][2])][routes[i-1][u][leg+1]] 
                 for u in route_len[i-1] for leg,_ in enumerate(routes[i-1][u][:-2])) +
              sum((T - mdl.getVarByName("Tarra" + str([i,u,len(routes[i-1][u][:-1])-2])) * mdl.getVarByName("B"+str([i,u]))
                  * dataset_airports["gr" + str(fleet[i-1][2])][routes[i-1][u][len(routes[i-1][u][:-1])-1]]) for u in route_len[i-1]) +
                T * (1 - sum(mdl.getVarByName("B"+str([i,u])) for u in route_len[i-1])) * 
                dataset_airports["gr" + str(fleet[i-1][2])][fleet[i-1][16]]
                 for i in N), name = "Grounding_costs");

In [20]:
mdl.params.NonConvex = 2
mdl.update()
mdl.optimize()

Parameter NonConvex unchanged
   Value: 2  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1111123 rows, 539762 columns and 2810661 nonzeros
Model fingerprint: 0x269019fd
Model has 85219 quadratic constraints
Model has 982784 general constraints
Variable types: 12910 continuous, 526852 integer (504432 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  QMatrix range    [1e-04, 2e+02]
  QLMatrix range   [1e-04, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
  QRHS range       [1e+03, 6e+06]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
--------------------------

Presolve removed 1678565 rows and 1372078 columns (presolve time = 105s) ...
Presolve removed 1680064 rows and 1395498 columns (presolve time = 260s) ...
Presolve removed 1833447 rows and 1437657 columns (presolve time = 260s) ...
Presolve removed 1833620 rows and 1466689 columns
Presolve time: 260.99s
Presolved: 3131 rows, 1908 columns, 9509 nonzeros
Presolved model has 430 SOS constraint(s)
Variable types: 0 continuous, 1908 integer (357 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.871720e+05   0.000000e+00   1014s
     301    0.0000000e+00   0.000000e+00   0.000000e+00   1014s

Root relaxation: objective 0.000000e+00, 301 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   16 18262.0000    0.00000   100%     - 1013s
H    0     0                       0

In [21]:
if mdl.status == GRB.INFEASIBLE:
    mdl.computeIIS()
    mdl.write("model.ilp")
    mdl.write("model.mps")